# Implementar un MLP con Backpropagation para resolver el problema de la función XOR 

<img src="figs/fig-MLP_XOR.png" width="40%">

# Crear la clase MLP

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Función de activación sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la sigmoide
def sigmoid_derivative(x):
    return x * (1 - x)

# Establece la semilla para la generación de números aleatorios
def seed(random_state=33):
    np.random.seed(random_state)

# Inicialización de javier
def xavier_initialization(input_size, output_size):
    # ¿En el parametro size es output, input?
    return np.random.normal(scale=np.sqrt(2 / (input_size + output_size)), size=(output_size, input_size))

# Inicialización normal
def normal_initialization(input_size, output_size):
    return np.random.randn(output_size, input_size) * 0.1

# Preprocesado de datos
def preprocesar(ruta):
    datos = pd.read_csv(ruta, header=0)
    datos_crudos = datos.to_numpy()

    x = datos_crudos[:, :-1]
    y = datos_crudos[:, -1:]

    return x, y

# Normalizar los datos
def normalizar_datos(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X)

# Crear mini lotes
def create_minibatches(X, y, batch_size):
    """
    Genera los lotes de datos (batchs) de acuerdo al parámetro batch_size de forma aleatoria para el procesamiento. 
    """
    n_samples = X.shape[0]
    indices = np.random.permutation(n_samples)  # Mezcla los índices aleatoriamente
    X_shuffled, y_shuffled = X[indices], y[indices]  # Reordena X e y según los índices aleatorios
    
    # Divide los datos en minibatches
    for X_batch, y_batch in zip(np.array_split(X_shuffled, np.ceil(n_samples / batch_size)), 
                                np.array_split(y_shuffled, np.ceil(n_samples / batch_size))):
        yield X_batch, y_batch


class MLP_TODO:
    def __init__(self, num_entradas, num_neuronas_ocultas, num_salidas, epochs, 
                 batch_size=128, learning_rate=0.2, random_state=42, initialization="xavier"):

        # Construcción
        seed(random_state)
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.error_mse = []
        self.accuracy_epoca = []
        
        # definir las capas
        if initialization == 'xavier':
            init_fun = xavier_initialization
        else : 
            init_fun = normal_initialization

        self.W1 = init_fun(num_entradas, num_neuronas_ocultas)
        self.b1 = np.zeros((1, num_neuronas_ocultas))
        self.W2 = init_fun(num_neuronas_ocultas, num_salidas)
        self.b2 = np.zeros((1, num_salidas))

    def forward(self, X):
        #----------------------------------------------
        # 1. Propagación hacia adelante (Forward pass)
        #----------------------------------------------
        self.X = X
        self.z_c1 = X @ self.W1.T + self.b1
        self.a_c1 = sigmoid(self.z_c1)
        self.z_c2 = self.a_c1 @ self.W2.T + self.b2
        y_pred = sigmoid(self.z_c2)  # Y^
        return y_pred

    def loss_function_MSE(self, y_pred, y):
        #----------------------------------------------
        # 2. Cálculo del error con MSE
        #----------------------------------------------
        self.y_pred = y_pred
        self.y = y
        error = 0.5 * np.mean((y_pred - y) ** 2)
        return error
    
    def backward(self):
        #----------------------------------------------
        # 3. Propagación hacia atrás (Backward pass)
        #----------------------------------------------
        
        #----------------------------------------------
        # Gradiente de la salida
        #----------------------------------------------
        dE_dy_pred = (self.y_pred - self.y) / self.y.shape[0] # Derivada del error respecto a la predicción con  N ejemplos
        d_y_pred_d_zc2 = sigmoid_derivative(self.y_pred)
        delta_c2 = dE_dy_pred * d_y_pred_d_zc2

        #----------------------------------------------
        # Gradiente en la capa oculta
        #----------------------------------------------
        # calcular la derivada de las suma ponderada respecto a las activaciones de la capa 1
        delta_c1 = (delta_c2 @ self.W2) * sigmoid_derivative(self.a_c1)

        #calcula el gradiente de pesos y bias
        self.dE_dW2 = delta_c2.T @ self.a_c1
        self.dE_db2 = np.sum(delta_c2, axis=0, keepdims=True)
        self.dE_dW1 = delta_c1.T @ self.X
        self.dE_db1 = np.sum(delta_c1, axis=0, keepdims=True)

    def update(self):  # Ejecución de la actualización de paramámetros
        #----------------------------------------------
        # Actualización de pesos de la capa de salida
        #---------------------------------------------- 
        
        self.W2 = self.W2 - self.learning_rate * self.dE_dW2 # Ojito con la T
        self.b2 = self.b2 - self.learning_rate * self.dE_db2

        #----------------------------------------------
        # Actuailzación de pesos de la capa oculta
        #----------------------------------------------
        #calcula el gradiente de la función de error respecto a los pesos de la capa 1
        self.W1 = self.W1 - self.learning_rate * self.dE_dW1
        self.b1 = self.b1 - self.learning_rate * self.dE_db1

    def predict(self, X):  # Predecir la categoría para datos nuevos
        y_pred = self.forward(X)
        # Obtener la clase para el clasificador binario
        y_pred = np.where(y_pred >= 0.5, 1, 0)
        return y_pred

    def train(self, X, Y):
        for epoch in range(self.epochs):

            num_batch = 0
            epoch_error  = 0

            # Procesamiento por lotes
            for X_batch, y_batch in create_minibatches(X, Y, self.batch_size):
                y_pred = self.forward(X_batch)
                error = self.loss_function_MSE(y_pred, y_batch)
                
                # if np.all(y_pred == Y) : aciertos += 1
                # self.accuracy_epoca.append(aciertos/epoch)

                epoch_error += error
                self.backward() # cálculo de los gradientes
                self.update() # actualización de los pesos y bias
                num_batch += 1
                # Imprimir el error cada N épocas
            
            # Almacena el error promedio por época
            self.error_mse.append(epoch_error/num_batch)

            # Obtener predicciones binarias para todo el conjunto de entrenamiento
            y_pred_total = self.predict(X)

            # Calcular la exactitud
            exactitud = self.calcular_accuracy(y_pred_total, Y) 
            
            # Almacenar la exactitud de la época
            self.accuracy_epoca.append(exactitud)

            #if epoch % 100 == 0: print(f"Época {epoch:05d} | MSE: {epoch_error/num_batch:.6f} | Exactitud: {exactitud:.4f}")

    def graficar(self, graficar_exactitud=True, guardar=True, nombre="grafica"):
        """ 
        Para MSE siempre se muestra 
        """
        # Preparar datos
        mse = np.arange(len(self.error_mse))

        # Crear tabla
        plt.figure(figsize=(10,6))

        #Graficar MSE
        plt.plot(mse, self.error_mse, label="MSE", color="red", linewidth=1)


        """ 
        Para la exactitud 
        """
        if graficar_exactitud and len(self.accuracy_epoca) > 0:
            accuracy = np.arange(len(self.accuracy_epoca))
            plt.plot(accuracy, self.accuracy_epoca, label="Exactitud", color="green", linewidth=1)
            plt.ylabel("MSE / Exactitud")
            titulo = "Evolución del Error (MSE) y Exactitud durante el entrenamiento"
        else:
            plt.ylabel("Error Cuadrático Medio (MSE)")
            titulo = "Evolución del Error (MSE) durante el entrenamiento"

        plt.title(titulo)
        plt.xlabel("Época")
        plt.legend()
        plt.grid(True, alpha=0.3)

        if guardar:
            plt.savefig(f'{nombre}.svg')
        plt.show()

    def calcular_accuracy(self, y_pred, y_verdadera):
        return np.mean(y_verdadera.flatten() == y_pred.flatten())

    def analizar(self, X, y):
        # Gráficar
        self.graficar(guardar=True)

        # Valores reales y predicción
        y_pred = self.predict(X)
        print(f"valores reales: {y.flatten()}")
        print(f"Predicciones  : {y_pred.flatten()}")

        # Calcular exactitud
        exactitud = self.calcular_accuracy(y_pred, y)
        print(f"Exactitud: {exactitud}")


# Entrenar para el ejemplo XOR

In [ ]:
#Ejemplo del uso para el entrenamiento

# Definimos los datos de entrada para XOR
X = np.array([[0, 0],[0, 1],[1, 0],[1, 1]])

# Salidas esperadas para XOR
y = np.array([[0],[1],[1],[0]])

entradas = 2
neuronas_ocultas = 2
salidas = 1
batch_size = 2 # Batch de 2 ejemplos # X.shape[0] # Batch de todos los ejemplos
# batch_size = X.shape[0] # Batch de todos los ejemplos
learning_rate = 1
epochs = 3000
clasificador_XOR = MLP_TODO(entradas, neuronas_ocultas, salidas, epochs=epochs, batch_size=batch_size, learning_rate=learning_rate)
clasificador_XOR.train(X,y)

clasificador_XOR.analizar(X, y)

# Preparar los datos para entrenar (Hiperparametros)

In [ ]:
from itertools import product
import pandas as pd

# Combinaciones
hiperparametros = {
    "num_neuronas_ocultas": [2, 4, 8, 16, 32, 128],
    "inicializacion": ["xavier", "normal"],
    "normalizacion_datos": [False, True],
    "learning_rate": [0.01, 0.1, 0.5],
    "batch_size": [8, 16, 32, 64],
    "dataset": ["iris_train.csv", "breast_cancer_test.csv", "wine_test.csv"]
}

# Lista de nombres de los parámetros
claves = list(hiperparametros.keys())
valores = list(hiperparametros.values())

# Generar combinaciones en una tupla
combinaciones = list(product(*valores))

# Crear una lista de diccionarios para acceder por nombre
dic_combinaciones = []
for valores_tupla in combinaciones:
    # Combina las claves con los valores de la tupla para formar un diccionario
    combinacion = dict(zip(claves, valores_tupla))
    dic_combinaciones.append(combinacion)

print(f"Número de combinaicones: {len(dic_combinaciones)}")
print(f"Combinaciones resultantes: {dic_combinaciones[0]}")

# Crear los modelos

In [ ]:
from itertools import product
import pandas as pd
import json

# Combinaciones
hiperparametros = {
    "num_neuronas_ocultas": [2, 4, 8, 16, 32, 128],
    "inicializacion": ["xavier", "normal"],  # Distribución Normal se mapea a 'normal'
    "normalizacion_datos": [False, True], # False = Sin normalizar, True = Normalización z-score
    "learning_rate": [0.01, 0.1, 0.5],
    "batch_size": [8, 16, 32, 64],
    "dataset": ["iris_train.csv", "breast_cancer_test.csv", "wine_test.csv"]
}

# Lista con nombre de los parametros
claves = list(hiperparametros.keys()) 
valores = list(hiperparametros.values())

# Generar combinaciones como tupla
combinaciones = list(product(*valores))

# Crear una lista de diccionarios para acceder por nombre
dict_combinaciones = []
for valores_tupla in combinaciones:
    combinacion = dict(zip(claves, valores_tupla))
    dict_combinaciones.append(combinacion)

"""
Preparar datos de entrenamiento
"""

epocas = 100
semilla = 33 
salidas = 1

# Ruta base de los datasets (basado en tu ejemplo anterior)
ruta = "./datasets/" 
resultado_data_set = {dataset: [] for dataset in hiperparametros["dataset"]}

# ¿Guardar solo el modelo o los parametros importantes?
modelos = []
resultados = []

# Entrenar

In [13]:
for indice, parametros in enumerate(dict_combinaciones):
    # Cargar y Preprocesar el Dataset
    nombre_data_set = parametros["dataset"]
    ruta_completa = ruta + parametros['dataset']
    
    X_crudo, Y = preprocesar(ruta_completa)

    # Normalizar datos
    if parametros["normalizacion_datos"]:
        X = normalizar_datos(X_crudo)
    else:
        X = X_crudo

    # Inicializar y entrenar el modelo
    num_entradas = X.shape[1]

    modelo = MLP_TODO(
        num_entradas=num_entradas,
        num_neuronas_ocultas=parametros['num_neuronas_ocultas'],
        num_salidas=salidas,
        epochs=epocas,
        batch_size=parametros['batch_size'],
        learning_rate=parametros['learning_rate'],
        random_state=semilla,
        initialization=parametros['inicializacion']
    )
    
    # modelos.append(modelo)
    modelo.train(X, Y)

    # Almacenar resultados
    exactitud_final = modelo.accuracy_epoca[-1]
    mse = modelo.error_mse[-1]
    
    # Guardar en un diccionario
    resultados.append({
        "modelo": parametros,
        "exactitud": exactitud_final,
        "mse": mse
    })
    # modelos.append(modelo)

nombre_datos = "resultados_entrenamiento.json"
try:
    with open(nombre_datos, "w") as file:
        json.dump(resultados, file, indent=4)
except IOError:
    print("No se pudo guardar el archivo")

/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264/3566135079.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/td/h2pqs96j2v5gv1p78d0qjz800000gn/T/ipykernel_13264

KeyboardInterrupt: 

In [ ]:
from itertools import product
import pandas as pd
import json

# NOTA: Asegúrate de que las funciones/clases (MLP_TODO, preprocesar, normalizar_datos)
# estén definidas o importadas en tu entorno.

# Combinaciones
hiperparametros = {
    "num_neuronas_ocultas": [2, 4, 8, 16, 32, 128],
    "inicializacion": ["xavier", "normal"],  # Distribución Normal se mapea a 'normal'
    "normalizacion_datos": [False, True], # False = Sin normalizar, True = Normalización z-score
    "learning_rate": [0.01, 0.1, 0.5],
    "batch_size": [8, 16, 32, 64],
    "dataset": ["iris_train.csv", "breast_cancer_test.csv", "wine_test.csv"]
}

# Generar la lista de diccionarios de combinaciones (Tu código es correcto)
claves = list(hiperparametros.keys()) 
valores = list(hiperparametros.values())
combinaciones = list(product(*valores))
dict_combinaciones = []
for valores_tupla in combinaciones:
    combinacion = dict(zip(claves, valores_tupla))
    dict_combinaciones.append(combinacion)

# Parámetros fijos
epocas = 100
semilla = 33 
salidas = 1
ruta = "./datasets/" 

# 🎯 CAMBIO CLAVE: Diccionario para agrupar resultados por nombre de dataset
# { "iris_train.csv": [ {modelo, exactitud, mse}, ... ], "breast_cancer_test.csv": [ ... ] }
resultados_por_dataset = {dataset: [] for dataset in hiperparametros["dataset"]} 

for indice, parametros in enumerate(dict_combinaciones):
    print(f"--- Ejecutando combinación {indice+1}/{len(dict_combinaciones)} en {parametros['dataset']} ---")
    
    # Cargar y Preprocesar el Dataset
    nombre_dataset = parametros['dataset'] # Guardamos el nombre aquí
    ruta_completa = ruta + nombre_dataset
    
    # ⚠️ ASUME que preprocesar/normalizar/MLP_TODO están definidos
    try:
        X_crudo, Y = preprocesar(ruta_completa)
    except FileNotFoundError:
        print(f"   ❌ Error: No se encontró el archivo: {ruta_completa}")
        continue # Saltar al siguiente
    
    # Normalizar datos
    if parametros["normalizacion_datos"]:
        X = normalizar_datos(X_crudo)
    else:
        X = X_crudo

    # Inicializar y entrenar el modelo
    num_entradas = X.shape[1]

    modelo = MLP_TODO(
        num_entradas=num_entradas,
        num_neuronas_ocultas=parametros['num_neuronas_ocultas'],
        num_salidas=salidas,
        epochs=epocas,
        batch_size=parametros['batch_size'],
        learning_rate=parametros['learning_rate'],
        random_state=semilla,
        initialization=parametros['inicializacion']
    )
    
    modelo.train(X, Y)

    # Almacenar resultados en la estructura deseada
    exactitud_final = modelo.accuracy_epoca[-1]
    mse = modelo.error_mse[-1]
    
    # 🎯 Creamos el objeto de resultado
    resultado_modelo = {
        "Configuracion": {k: v for k, v in parametros.items() if k != 'dataset'}, # Quitar 'dataset' de la configuración
        "exactitud": exactitud_final,
        "mse": mse
    }
    
    # 🎯 Agregamos a la lista correcta según el nombre del dataset
    resultados_por_dataset[nombre_dataset].append(resultado_modelo)


# --- Guardar resultados en JSON separados por dataset ---

for nombre_dataset, resultados_lista in resultados_por_dataset.items():
    # Creamos el nombre del archivo basado en el dataset (ej: resultados_iris_train.json)
    nombre_archivo = f"resultados_{nombre_dataset.replace('.csv', '')}.json"
    
    try:
        with open(nombre_archivo, "w") as file:
            json.dump(resultados_lista, file, indent=4)
        print(f"\n✅ Resultados para '{nombre_dataset}' guardados en: {nombre_archivo}")
    except IOError:
        print(f"\n❌ No se pudo guardar el archivo: {nombre_archivo}")

In [15]:
import pandas as pd

resultados_json = pd.read_json("./resultados_entrenamiento.json")
resultados_orden = (resultados_json.sort_values(by=['exactitud', "mse"], ascending=[False, True]))
resultados_orden.head(30)

,modelo,exactitud,mse
780,"{'num_neuronas_ocultas': 128, 'inicializacion'...",1.0,0.000385
852,"{'num_neuronas_ocultas': 128, 'inicializacion'...",1.0,0.000395
636,"{'num_neuronas_ocultas': 32, 'inicializacion':...",1.0,0.000445
708,"{'num_neuronas_ocultas': 32, 'inicializacion':...",1.0,0.000479
744,"{'num_neuronas_ocultas': 128, 'inicializacion'...",1.0,0.000503
492,"{'num_neuronas_ocultas': 16, 'inicializacion':...",1.0,0.000504
816,"{'num_neuronas_ocultas': 128, 'inicializacion'...",1.0,0.000519
564,"{'num_neuronas_ocultas': 16, 'inicializacion':...",1.0,0.000528
600,"{'num_neuronas_ocultas': 32, 'inicializacion':...",1.0,0.000564
672,"{'num_neuronas_ocultas': 32, 'inicializacion':...",1.0,0.000597
